In [2]:
import os

import numpy as np
import pandas as pd
import simplstyles
import matplotlib.pyplot as plt
import seaborn as sns
import voxelwise_tutorials.viz as viz
from himalaya.backend import set_backend
from matplotlib import pyplot as plt
from scipy.stats import zscore
from sklearn.pipeline import make_pipeline

from compare_variance_residual.stimuli_utils.TemporalChunkSplitter import TemporalChunkSplitter

In [2]:
def get_result_path(modality, subject):
    path = os.path.join("results", modality, f"subject{subject:02}")
    os.makedirs(path, exist_ok=True)
    return path

In [4]:
plt.style.use('nord-light-talk')
data_dir = "../data"
backend = set_backend('torch_cuda', on_error='warn')

In [4]:
language_model = "bert-base"
layer = 9
num_layers = 12
feature = "semantic"
modality = "reading"
subject = 1
low_level_feature = "letters"
trim = 5  # remove 5 TRs from the start and end of each story
sequence_length = 20
number_of_delays = 4

In [5]:
alphas = np.logspace(-5, 5, 10)
cv = 10
n_iter = 10

# Load Features

In [6]:
import h5py

features_train = h5py.File(os.path.join(data_dir, 'features', 'features_trn_NEW.hdf'), 'r')
features_val = h5py.File(os.path.join(data_dir, 'features', 'features_val_NEW.hdf'), 'r')
print(features_train.keys(), features_val.keys())
print(features_train['story_01'].keys())

<KeysViewHDF5 ['story_01', 'story_02', 'story_03', 'story_04', 'story_05', 'story_06', 'story_07', 'story_08', 'story_09', 'story_10']> <KeysViewHDF5 ['story_11']>
<KeysViewHDF5 ['english1000', 'letters', 'numletters', 'numphonemes', 'numwords', 'pauses', 'phonemes', 'word_length_std']>


## Load Low Level Feature

In [7]:
low_level_train = np.vstack([zscore(features_train[story][low_level_feature]) for story in features_train.keys()])
low_level_val = np.vstack([zscore(features_val[story][low_level_feature]) for story in features_val.keys()])
low_level_train, low_level_val = np.nan_to_num(low_level_train), np.nan_to_num(low_level_val)
print(low_level_train.shape, low_level_val.shape)

(3887, 26) (306, 26)


## Load High Level (NLP) Features

In [8]:
# downsampled_embeddings = np.load(f"../{language_model}{sequence_length}_downsampled.npy", allow_pickle=True)
# print(downsampled_embeddings.item().keys())

dict_keys(['alternateithicatom', 'avatar', 'howtodraw', 'legacy', 'life', 'myfirstdaywiththeyankees', 'naked', 'odetostepfather', 'souls', 'undertheinfluence', 'wheretheressmoke'])


In [9]:
# Rstories = ['alternateithicatom', 'avatar', 'howtodraw', 'legacy',
#             'life', 'myfirstdaywiththeyankees', 'naked',
#             'odetostepfather', 'souls', 'undertheinfluence']
# Pstories = ['wheretheressmoke']
#
# semantic_embeddings_train = np.vstack([zscore(downsampled_embeddings.item()[story][layer]) for story in Rstories])
# semantic_embeddings_val = np.vstack([zscore(downsampled_embeddings.item()[story][layer]) for story in Pstories])
# semantic_embeddings_train, semantic_embeddings_val = np.nan_to_num(semantic_embeddings_train), np.nan_to_num(
#     semantic_embeddings_val)
# print(semantic_embeddings_train.shape, semantic_embeddings_val.shape)

(3887, 768) (306, 768)


In [9]:
semantic_train = np.vstack([zscore(features_train[story]['english1000']) for story in features_train.keys()])
semantic_val = np.vstack([zscore(features_val[story]['english1000']) for story in features_val.keys()])
print(semantic_train.shape, semantic_val.shape)

(3887, 985) (306, 985)


# Load Brain Data

In [10]:
from voxelwise_tutorials.io import load_hdf5_array

Y_train_filename = os.path.join(data_dir, 'responses', f'subject{subject:02}_{modality}_fmri_data_trn.hdf')
Y_train = load_hdf5_array(Y_train_filename)

Y_test_filename = os.path.join(data_dir, 'responses', f'subject{subject:02}_{modality}_fmri_data_val.hdf')
Y_test = load_hdf5_array(Y_test_filename)

Y_train = np.vstack([zscore(Y_train[story][:-trim]) for story in Y_train.keys()])
Ys_test = [np.vstack([zscore(Y_test[story][i][:-trim]) for story in Y_test.keys()]) for i in range(2)]

print(Y_train.shape, np.array(Ys_test).shape)
Y_train, Ys_test = np.nan_to_num(Y_train), np.nan_to_num(Ys_test)

/tmp/ipykernel_62600/893258922.py:9: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  Y_train = np.vstack([zscore(Y_train[story][:-trim]) for story in Y_train.keys()])
/tmp/ipykernel_62600/893258922.py:9: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  Y_train = np.vstack([zscore(Y_train[story][:-trim]) for story in Y_train.keys()])
/tmp/ipykernel_62600/893258922.py:9: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  Y_train = np.vstack([zscore(Y_train[story][:-trim]) for story in Y_train.keys()])
/tmp/ipykernel_62600/893258922.py:9: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cance

(3887, 81133) (2, 306, 81133)


## convert everything to float32

In [11]:
low_level_train = low_level_train.astype(np.float32)
low_level_val = low_level_val.astype(np.float32)
semantic_train = semantic_train.astype(np.float32)
semantic_val = semantic_val.astype(np.float32)
Y_train = Y_train.astype(np.float32)
Ys_test = [Y_test.astype(np.float32) for Y_test in Ys_test]

# Variance Partitioning

## Low Level Prediction

In [12]:
low_level_file = os.path.join(get_result_path(modality, subject), f"vp_low_level_{low_level_feature}_scores.csv")

In [13]:
from himalaya.ridge import RidgeCV
from voxelwise_tutorials.delayer import Delayer

if not os.path.exists(low_level_file):
    print(f"Saving {low_level_file}")
    # delay stimuli to account for hemodynamic lag
    delays = range(1, number_of_delays + 1)
    delayer = Delayer(delays=delays)
    pipeline = make_pipeline(
        delayer,
        RidgeCV(
            alphas=alphas, cv=cv,
            solver_params=dict(n_targets_batch=50, n_alphas_batch=1, n_targets_batch_refit=50)
        )
    )

    pipeline.fit(low_level_train, Y_train)
    vp_low_level_scores = []
    for Y_test in Ys_test:
        vp_low_level_scores.append(pipeline.score(low_level_val, Y_test))
    vp_low_level_scores = np.array(vp_low_level_scores)
    # save as csv
    vp_low_level_scores = pd.DataFrame(vp_low_level_scores)
    vp_low_level_scores.to_csv(low_level_file)
else:
    print(f"Loading {low_level_file}")
    vp_low_level_scores = pd.read_csv(low_level_file, index_col=0).values
print(vp_low_level_scores.max(), vp_low_level_scores.min(), vp_low_level_scores.mean())

Loading results/reading/subject01/vp_low_level_letters_scores.csv
0.3245427479184268 -0.0631824860921701 0.0023216666772059953


## Semantic Prediction

In [14]:
semantic_file = os.path.join(get_result_path(modality, subject), f"vp_semantic_{layer:02}_scores.csv")

In [15]:
from himalaya.ridge import RidgeCV
from voxelwise_tutorials.delayer import Delayer

if not os.path.exists(semantic_file):
    print(f"Saving {semantic_file}")
    # delay stimuli to account for hemodynamic lag
    delays = range(1, number_of_delays + 1)
    delayer = Delayer(delays=delays)
    pipeline = make_pipeline(
        delayer,
        RidgeCV(
            alphas=alphas, cv=cv,
            solver_params=dict(n_targets_batch=50, n_alphas_batch=1, n_targets_batch_refit=50)
        )
    )

    pipeline.fit(semantic_train, Y_train)
    vp_semantic_scores = []
    for Y_test in Ys_test:
        vp_semantic_scores.append(pipeline.score(semantic_val, Y_test))
    # save as csv
    vp_semantic_scores = pd.DataFrame(
        {'vp_semantic_score_0': vp_semantic_scores[0], 'vp_semantic_score_1': vp_semantic_scores[1]})
    vp_semantic_scores.to_csv(semantic_file)
else:
    print(f"Loading {semantic_file}")
    vp_semantic_scores = pd.read_csv(semantic_file, index_col=0).values
print(vp_semantic_scores.max(), vp_semantic_scores.min(), vp_semantic_scores.mean())

Loading results/reading/subject01/vp_semantic_09_scores.csv
0.3457906230440486 -0.1617008729669207 0.0023683850304021454


## Joint Prediction

In [16]:
joint_file = os.path.join(get_result_path(modality, subject), f"vp_joint_{feature}_{low_level_feature}_scores.csv")

In [24]:
from himalaya.ridge import BandedRidgeCV, ColumnTransformerNoStack

if not os.path.exists(joint_file):
    print(f"Saving {joint_file}")
    # delay stimuli to account for hemodynamic lag
    delays = range(1, number_of_delays + 1)
    delayer = Delayer(delays=delays)

    start_and_end = np.concatenate([[0], np.cumsum([semantic_train.shape[1], low_level_train.shape[1]])])
    slices = [
        slice(start, end)
        for start, end in zip(start_and_end[:-1], start_and_end[1:])
    ]
    print(slices)
    ct = ColumnTransformerNoStack(transformers=[('semantic', delayer, slices[0]), ('low_level', delayer, slices[1])])

    pipeline = make_pipeline(
        ct,
        BandedRidgeCV(
            cv=TemporalChunkSplitter(), groups="input",
            solver_params=dict(n_iter=n_iter, alphas=alphas, n_targets_batch=500, n_alphas_batch=5,
                               n_targets_batch_refit=100)
        )
    )

    X_train = np.concatenate([semantic_train, low_level_train], axis=1)
    X_test = np.concatenate([semantic_val, low_level_val], axis=1)
    pipeline.fit(X_train, Y_train)

    vp_joint_scores = []
    for Y_test in Ys_test:
        vp_joint_scores.append(pipeline.score(X_test, Y_test))
    vp_joint_scores = vp_joint_scores.cpu()
    # save as csv
    vp_joint_scores = pd.DataFrame({'vp_joint_score_0': vp_joint_scores[0], 'vp_joint_score_1': vp_joint_scores[1]})
    vp_joint_scores.to_csv(joint_file)
else:
    print(f"Loading {joint_file}")
    vp_joint_scores = pd.read_csv(joint_file, index_col=0).values
print(vp_joint_scores.max(), vp_joint_scores.min(), vp_joint_scores.mean())

Loading results/reading/subject01/vp_joint_semantic_letters_scores.csv
0.3585633 -0.11069548 0.004417512358557558


# Residual Method

# Plot Brain Maps

In [ ]:
def plot_flatmap_from_mapper(voxels, mapper_file, ax=None, alpha=0.7, cmap=plt.get_cmap(), vmin=None, vmax=None,
                             with_curvature=True, with_rois=True, with_colorbar=True,
                             colorbar_location=(.4, .9, .2, .05)):
    """Plot a flatmap from a mapper file, with 1D data.

    This function is equivalent to the pycortex functions:
    cortex.quickshow(cortex.Volume(voxels, ...), ...)

    Note that this function does not have the full capability of pycortex,
    since it is based on flatmap mappers and not on the original brain
    surface of the subject.

    Parameters
    ----------
    voxels : array of shape (n_voxels, )
        Data to be plotted.
    mapper_file : str
        File name of the mapper.
    ax : matplotlib Axes or None.
        Axes where the figure will be plotted.
        If None, a new figure is created.
    alpha : float in [0, 1], or array of shape (n_voxels, )
        Transparency of the flatmap.
    cmap : str
        Name of the matplotlib colormap.
    vmin : float or None
        Minimum value of the colormap. If None, use the 1st percentile of the
        `voxels` array.
    vmax : float or None
        Minimum value of the colormap. If None, use the 99th percentile of the
        `voxels` array.
    with_curvature : bool
        If True, show the curvature below the data layer.
    with_rois : bool
        If True, show the ROIs labels above the data layer.
    colorbar_location : [left, bottom, width, height]
        Location of the colorbar. All quantities are in fractions of figure
        width and height.

    Returns
    -------
    ax : matplotlib Axes
        Axes where the figure has been plotted.
    """
    # create a figure
    if ax is None:
        flatmap_mask = load_hdf5_array(mapper_file, key='flatmap_mask')
        figsize = np.array(flatmap_mask.shape) / 100.
        fig = plt.figure(figsize=figsize)
        ax = fig.add_axes((0, 0, 1, 1))
        ax.axis('off')

    # process plotting parameters
    vmin = np.percentile(voxels, 1) if vmin is None else vmin
    vmax = np.percentile(voxels, 99) if vmax is None else vmax
    if isinstance(alpha, np.ndarray):
        alpha = viz.map_voxels_to_flatmap(alpha, mapper_file)

    # plot the data
    image = viz.map_voxels_to_flatmap(voxels, mapper_file)
    cimg = ax.imshow(image, aspect='equal', zorder=1, alpha=alpha, cmap=cmap,
                     vmin=vmin, vmax=vmax)

    if with_colorbar:
        try:
            cbar = ax.inset_axes(colorbar_location)
        except AttributeError:  # for matplotlib < 3.0
            cbar = ax.figure.add_axes(colorbar_location)
        colorbar = ax.figure.colorbar(cimg, cax=cbar, orientation='horizontal')
        colorbar.ax.set_title("Pearson Correlation of Y_true and Y_pred", fontsize=14)

    # plot additional layers if present
    viz._plot_addition_layers(ax=ax, n_voxels=voxels.shape[0],
                              mapper_file=mapper_file,
                              with_curvature=with_curvature, with_rois=with_rois)

    return ax

In [ ]:
path = get_path(language_model, feature, modality, subject, low_level_feature, layer)
correlation = np.nan_to_num(np.load(path, allow_pickle=True))
mapper_path = os.path.join("../data", 'mappers', f"subject{subject:02}_mappers.hdf")
flatmap_mask = load_hdf5_array(mapper_path, key='flatmap_mask')

In [ ]:
figsize = np.array(flatmap_mask.shape) / 100.
fig = plt.figure(figsize=figsize)
ax = fig.add_axes((0, 0, 1, 1))
ax.axis('off')
plot_flatmap_from_mapper(correlation, mapper_path, ax=ax, with_curvature=False, alpha=1, vmin=0,
                         # vmin=np.min(correlation),
                         vmax=np.max(correlation), colorbar_location=[0.75, 0.05, 0.2, 0.05])
plt.show()